In [ ]:
from sklearn.impute import KNNImputer
import pandas as pd
import numpy as np
# from missingpy import MissForest
import matplotlib.pylab as plt
import seaborn as sns

In [ ]:
# a dictionary to put all of the accuracies
accuracies = {}

In [ ]:
# loading data with 10% missing
gse111631_10 = pd.read_csv('/content/data/Shareddrives/Deep_Learning_G&R/gse111631_10.csv').rename(columns={'Unnamed: 0':'Sample'}).set_index('Sample')
print(gse111631_10.isna().sum().sum()/(gse111631_10.shape[0]*gse111631_10.shape[1])*100, '% missing values')
print(gse111631_10.shape)
gse111631_10.head()

10.252032520325203 % missing values
(123, 100)


,cg17357548,cg24592462,cg07068870,cg07576175,cg04384867,cg06293982,cg07452706,cg09622586,cg19249622,cg26688435,...,cg01021551,cg13299707,cg23355015,cg26557696,cg00942293,cg23842941,cg17573068,cg11070059,cg27352639,cg01744133
Sample,,,,,,,,,,,,,,,,,,,,,
201172560029_R04C01,0.945,0.014,0.066,0.022,NaN,0.940,0.300,NaN,0.972,0.964,...,0.016,0.858,0.976,0.896,0.870,NaN,0.015,0.159,0.045,0.048
201172560029_R05C01,0.949,0.013,0.048,0.026,0.957,0.930,0.352,0.957,0.973,0.960,...,0.015,0.776,0.973,0.922,0.898,0.736,0.017,0.158,0.063,0.055
201172560029_R06C01,0.932,0.013,0.051,0.020,0.964,0.936,0.315,0.964,0.970,0.963,...,NaN,0.795,0.977,0.930,0.891,0.741,0.016,0.181,0.065,0.043
201172560029_R07C01,0.951,0.011,NaN,0.029,0.967,NaN,0.242,0.957,0.953,0.966,...,0.016,NaN,0.972,0.927,NaN,0.755,0.018,0.159,0.066,0.055
201172560029_R08C01,0.932,0.011,0.062,0.025,0.970,0.939,0.335,0.955,0.969,0.942,...,0.019,0.866,0.979,0.855,0.884,0.788,0.016,0.237,0.043,0.048


In [ ]:
# loading original
original_gse111631 = pd.read_csv('/content/data/Shareddrives/Deep_Learning_G&R/gse111631_original.csv').rename(columns={'Unnamed: 0':'Sample'}).set_index('Sample')
print(original_gse111631.shape)
original_gse111631.head()

(123, 100)


,cg17357548,cg24592462,cg07068870,cg07576175,cg04384867,cg06293982,cg07452706,cg09622586,cg19249622,cg26688435,...,cg01021551,cg13299707,cg23355015,cg26557696,cg00942293,cg23842941,cg17573068,cg11070059,cg27352639,cg01744133
Sample,,,,,,,,,,,,,,,,,,,,,
201172560029_R04C01,0.945,0.014,0.066,0.022,0.973,0.940,0.300,0.929,0.972,0.964,...,0.016,0.858,0.976,0.896,0.870,0.751,0.015,0.159,0.045,0.048
201172560029_R05C01,0.949,0.013,0.048,0.026,0.957,0.930,0.352,0.957,0.973,0.960,...,0.015,0.776,0.973,0.922,0.898,0.736,0.017,0.158,0.063,0.055
201172560029_R06C01,0.932,0.013,0.051,0.020,0.964,0.936,0.315,0.964,0.970,0.963,...,0.015,0.795,0.977,0.930,0.891,0.741,0.016,0.181,0.065,0.043
201172560029_R07C01,0.951,0.011,0.068,0.029,0.967,0.922,0.242,0.957,0.953,0.966,...,0.016,0.836,0.972,0.927,0.880,0.755,0.018,0.159,0.066,0.055
201172560029_R08C01,0.932,0.011,0.062,0.025,0.970,0.939,0.335,0.955,0.969,0.942,...,0.019,0.866,0.979,0.855,0.884,0.788,0.016,0.237,0.043,0.048


In [ ]:
def get_accuracies(imputed, missing=gse111631_10, original=original_gse111631):
    # sum of squared errors, mean of average absolute errors, root mean squared error
    ret = {}
    ret['sse'] = ((imputed.mask(~missing.isna()) - original.mask(~missing.isna()))**2).sum().sum()
    ret['mae'] = (imputed.mask(~missing.isna()) - original.mask(~missing.isna())).abs().sum().mean()
    ret['rmse'] = (((imputed.mask(~missing.isna()) - original.mask(~missing.isna()))**2).sum().mean())**(1/2)
    return ret

**DeepImpute**

In [ ]:
!pip install deepimpute

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from deepimpute.multinet import MultiNet
import pandas as pd

data = original_gse111631

data = data * 100
print('Working on {} cells and {} genes'.format(*data.shape))


Working on 123 cells and 100 genes


In [ ]:
from deepimpute.multinet import MultiNet

imputed_x = imputed * 100
model = MultiNet()
model.fit(data)
imputed = model.predict(gse111631_10)



Using all the cores (2)
Input dataset is 123 cells (rows) and 100 genes (columns)
First 3 rows and columns:
                     cg17357548  cg24592462  cg07068870
Sample                                                 
201172560029_R04C01        94.5         1.4         6.6
201172560029_R05C01        94.9         1.3         4.8
201172560029_R06C01        93.2         1.3         5.1
512 genes selected for imputation
Net 0: 0 predictors, 512 targets
Normalization
Building network
[{'type': 'dense', 'neurons': 256, 'activation': 'relu'}, {'type': 'dropout', 'rate': 0.2}]
Fitting with 123 cells
Epoch 1/500


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


2/2 [==============================] - 1s 154ms/step - loss: 46.7882 - val_loss: 46.8292
Epoch 2/500
2/2 [==============================] - 0s 33ms/step - loss: 46.7856 - val_loss: 46.8267
Epoch 3/500
2/2 [==============================] - 0s 30ms/step - loss: 46.7831 - val_loss: 46.8241
Epoch 4/500
2/2 [==============================] - 0s 43ms/step - loss: 46.7805 - val_loss: 46.8215
Epoch 5/500
2/2 [==============================] - 0s 32ms/step - loss: 46.7780 - val_loss: 46.8190
Epoch 6/500
2/2 [==============================] - 0s 32ms/step - loss: 46.7754 - val_loss: 46.8164
Epoch 7/500
2/2 [==============================] - 0s 34ms/step - loss: 46.7729 - val_loss: 46.8139
Epoch 8/500
2/2 [==============================] - 0s 40ms/step - loss: 46.7703 - val_loss: 46.8113
Epoch 9/500
2/2 [==============================] - 0s 32ms/step - loss: 46.7678 - val_loss: 46.8088
Epoch 10/500
2/2 [==============================] - 0s 31ms/step - loss: 46.7652 - val_loss: 46.8062
Epoch 11/5

In [ ]:
def get_accuracies(imputed, missing, original):
    # sum of squared errors, mean of average absolute errors, root mean squared error
    ret = {}
    ret['sse'] = ((imputed.mask(~missing.isna()) - original.mask(~missing.isna()))**2).sum().sum()/100
    ret['mae'] = (imputed.mask(~missing.isna()) - original.mask(~missing.isna())).abs().sum().mean()/10
    ret['rmse'] = (((imputed.mask(~missing.isna()) - original.mask(~missing.isna()))**2).sum().mean())**(1/2)/10
    return ret

i = get_accuracies(imputed, gse111631_10, original_gse111631)


print(imputed.head())
i


                     cg17357548  cg24592462  cg07068870  cg07576175  \
Sample                                                                
201172560029_R04C01       0.945       0.014    0.066000       0.022   
201172560029_R05C01       0.949       0.013    0.048000       0.026   
201172560029_R06C01       0.932       0.013    0.051000       0.020   
201172560029_R07C01       0.951       0.011    1.105475       0.029   
201172560029_R08C01       0.932       0.011    0.062000       0.025   

                     cg04384867  cg06293982  cg07452706  cg09622586  \
Sample                                                                
201172560029_R04C01    1.106017    0.940000       0.300    1.106015   
201172560029_R05C01    0.957000    0.930000       0.352    0.957000   
201172560029_R06C01    0.964000    0.936000       0.315    0.964000   
201172560029_R07C01    0.967000    1.106015       0.242    0.957000   
201172560029_R08C01    0.970000    0.939000       0.335    0.955000   

    

{'sse': 4.1464459665701305,
 'mae': 0.5437117257108689,
 'rmse': 0.20362823887099085}